<h3>1. Проверка на четность</h3>
    Задача: На языке Python или С/С++, написать алгоритм (функцию) определения четности целого числа, который будет аналогичен нижеприведенному по функциональности, но отличен по своей сути.
Объяснить плюсы и минусы обеих реализаций.
    def isEven(value):return value%2==0
    Можно воспользоваться фактом, что в двоичной записи четные числа оканчиваются на 0, нечетные на 1 (функция isEven_1). Минусом метода является то, что последняя двоичная цифра автоматически преобразуется в string, который затем необходимо преобразовать в int, а затем в bool. Сам тип string тяжелый т.к. Unicode и первое преобразование вычислительно сложное. 
    Более правильным способом будет использование побитового И c единицей (функция isEven_2), что, по сути, даст тот же результат, но без лишних преобразований. В более низкоуровневом, компилируемом языке это была бы очень быстрая операция, условно говоря, на один такт процессора. 

In [57]:
N = 100000
import time



def test(func,N):
    #сначала проверим, что функция отрабатывает правильно
    for i in range(N):
        if isEven(i)!=func(i):
            print("ERROR at {}, {}!={}".format(i,isEven(i),func(i)))
            break
    else:
        #теперь в отдельном цикле измерим время
        start = time.time()
        for i in range(N):
            func(i)
        end = time.time()
        print("Test OK, time={}".format(end-start))

def isEven(value):
    return value%2==0

def isEven_1(value):
    return not(int(bin(value)[-1]))

def isEven_2(value):
    return not(value & 1)


In [60]:
print(test(isEven,N),test(isEven_1,N),test(isEven_2,N),sep="/n")

Test OK, time=0.012027502059936523
Test OK, time=0.040644168853759766
Test OK, time=0.01196908950805664
None/nNone/nNone


как можно заметить, isEven_2 работает чуть быстрее. Скорее всего, в компилируемом языке разница была бы более заметна.

<h3> 2. На языках Python(2.7) и/или С++, написать минимум по 2 класса реализовывающих циклический буфер. </h3>

1.1 "Низкоуровневая" реализация

In [128]:
class RingBuffer_1(object):
    def __init__(self,size):
        self._data = bytearray(size) #Массив байт
        self.head = 0                #смещение записи
        self.tail = 0                #смещение чтения
        self.flgEmpty=True           #флаг, что буффер пуст
        self.flgFull=False           #флаг, что буффер полон
        
    #записать один байт 
    #------------------------------------------------------------------------------------------------
    def _push(self,value):
        #если передан символ. Можно также определить поведение для других объектов
        if isinstance(value,str):
            value =  ord(value)
        
        try:
            if value//256>0:   #если передано число больше байта
                self._push(value%256)
                value = value//256
                
            self._data[self.head]=value  #записываем число
            self.head=(self.head+1)%len(self._data) #сдвигаем указатель
            if self.head==self.tail:
                self.flgFull=True
            if self.flgFull:
                self.tail=self.head #если начало "догнало" конец, сдвигаем конец
            self.flgEmpty=False
        except (ValueError):
            print("Value error, value must be byte")
            
            
    #записать что-то произвольное в произвольном кол-ве
    #--------------------------------------------------------------------------------------------------
    def push(self,*args):    
        for values in args:
            try:
                for value in values:
                    self._push(value)
            except (TypeError):
                self._push(values)
     
    
    #прочитать один байт
    #--------------------------------------------------------------------------------------------------
    def _pop(self):
        if (not(self.flgFull) and (self.head==self.tail)): #если буффер опустошен
            self.flagEmpty=True
            return 256
        tmp = self.tail
        self.tail=(self.tail+1)%len(self._data) #сдвигаем указатель чтения
        self.flgFull=False
        return self._data[tmp]  
    
    #прочитать произвольное кол-во байт
    #---------------------------------------------------------------------------------------------------
    def pop(self,size):
        res = []
        if(size>0):
            for i in range(size):
                res.append(self._pop())
                if res[-1]==256:
                    print("Extracted {} byte(s), then buffer got empty".format(i))
                    res.pop()
                    break
            return res
        else:
            print("Size must be>0")
        
    #удобный просмотр содержимого
    #
    def __repr__(self):
        res = ''
        for i in range(len(self._data)):
            res+="  {}  ".format(self._data[i].__repr__())
        return res
    

In [129]:
b = RingBuffer_1(5)
b.push('abc',0,[1,2])
print(b)
print(b.pop(2))

  2    98    99    0    1  
[98, 99]


2.2 С помощью numpy.roll. Позиция считывания всегда current_size-1, позиция записи всегда 0. Буффер принимает тольно числа, т.к. реализован на базе ndarray. При желании можно реализовать преобразование произвольного класса в число и считывание/запись произвольного кол-ва объектов аналогично предыдущему классу.

In [217]:
import numpy as np
class RingBuffer_2(object):
    def __init__(self,size):
        self._data = np.zeros(size)
        self._curr_size = 0
    
    def __repr__(self):
        return self._data.__repr__()
    
    def push(self,value):
        if (self._data.size>self._curr_size):
            self._curr_size +=1
        self._data = np.roll(self._data,1)
        self._data[0] = value
                
    def pop(self):
        if (self._curr_size == 0):
            print("Buffer empty")
            return
        tmp = self._data[self._curr_size-1]
        self._curr_size-=1
        return(tmp)



In [218]:
b = RingBuffer_2(5)



In [219]:
for i in range(6):
    b.push(i+1)
    print(b)

array([1., 0., 0., 0., 0.])
array([2., 1., 0., 0., 0.])
array([3., 2., 1., 0., 0.])
array([4., 3., 2., 1., 0.])
array([5., 4., 3., 2., 1.])
array([6., 5., 4., 3., 2.])


In [220]:
for i in range(6):
    print(b.pop())

2.0
3.0
4.0
5.0
6.0
Buffer empty
None


С помощью 2-х классов, т.е. заполнившись, класс меняет поведение

In [ ]:
class RingBuffer_3:
    def __init__(self,size_max):
        self.max = size_max
        self.data = []
        
    def append(self,x):
        self.data.append(x)
        if len(self.data) == self.max:
            self.cur=0
            self.__class__ = RingBufferFull
            
    def get(self):
        return self.data


class RingBuffer_3_Full:

    def append(self,x):
        self.data[self.cur]=x
        self.cur=(self.cur+1) % self.max
        
    def get(self):
        return self.data[self.cur:]+self.data[:self.cur]

Можно также реализовать с помощь модуля collections и встроенного класса deque, аналогично numpy.

<h3>3. На языке Python или С/С++, написать функцию, которая быстрее всего (по процессорным тикам) отсортирует данный ей массив чисел.</h3>

Массив может быть любого размера со случайным порядком чисел (в том числе и отсортированным).

In [261]:
def test(func,sample):
    import time
    start=time.time()
    func(sample)
    end=time.time()
    print(end-start)
    

def generate_sample(N):
    from random import randint
    return [randint(1, 99) for _ in range(N)]


def BubbleSort(a):
    n = len(a)
    for i in range(n-1):
        for j in range(0, n-i-1):
            if a[j] > a[j + 1] :
                a[j], a[j + 1] = a[j + 1], a[j]

def ShakerSort(a): 
    upper = len(a) - 1
    lower = 0
    no_swap = False
    while (not no_swap and upper - lower > 1):
        no_swap = True
        for j in range(lower, upper):
            if a[j + 1] < a[j]:
                a[j + 1], a[j] = a[j], a[j + 1]
                no_swap = False
        upper = upper - 1
        for j in range(upper, lower, -1):
            if a[j - 1] > a[j]:
                a[j - 1], a[j] = a[j], a[j - 1]
                no_swap = False
        lower = lower + 1
    return a
    
def InsertionSort(a):
    for i in range(len(a)):
        x = a[i];
        j = i;
        while (j > 0 and a[j - 1] > x):
            a[j] = a[j - 1];
            j-=1
        a[j] = x;
    return a

def QuickSort(a):
    def partition(a, low, high):
        i = (low-1)        
        pivot = a[high]    
        for j in range(low, high):
            if a[j] <= pivot:
                i = i+1
                a[i], a[j] = a[j], a[i]
        a[i+1], a[high] = a[high], a[i+1]
        return (i+1)
    
    def quickSort(a, low, high):
        if len(a) == 1:
            return a
        if low < high:
            pi = partition(a, low, high)
            quickSort(a, low, pi-1)
            quickSort(a, pi+1, high)
        return a
    
    return quickSort(a,0,len(a)-1)

def MergeSort(a):
    if len(a) > 1:
        mid = len(a)//2
        L = a[:mid]
        R = a[mid:]
        mergeSort(L)
        mergeSort(R)
        i = j = k = 0
        while i < len(L) and j < len(R):
            if L[i] < R[j]:
                a[k] = L[i]
                i += 1
            else:
                a[k] = R[j]
                j += 1
            k += 1
        while i < len(L):
            a[k] = L[i]
            i += 1
            k += 1
        while j < len(R):
            a[k] = R[j]
            j += 1
            k += 1
    return a

In [262]:
funcs = [BubbleSort,ShakerSort,InsertionSort,QuickSort,MergeSort,sorted]
orig_sample = generate_sample(10000)
for func in funcs:
    sample = orig_sample.copy()
    test(func,sample)

7.904202222824097
6.756250619888306
4.589479684829712
0.09378480911254883
0.04687213897705078
0.0


Скорость сортировки сильно зависит от того, насколько массив изначально близок к отсортированному. QuickSort в лучшем случае сотрирует за O(n), в худшем за O(n^2), Merge sort за O(n log n). Интересно, что встроенная функция sorted работает быстрее всего. 